# Sensitivity Analysis
This notebook analyzes sensitivity datasets to explore correlations between rankability and sensitivity.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
from scipy import stats
from sklearn.linear_model import LinearRegression

In [ ]:
DATA_CSV_NAME = "sensitivity_dataset_real.csv"

In [ ]:
data = pd.read_csv(DATA_CSV_NAME)
len(data)

In [ ]:
list(data.columns)

In [ ]:
sensitivities = [col for col in data.columns if "mean_sensitivity" in col]
sensitivities

In [ ]:
data["overall_mean_sensitivity"] = data[sensitivities].mean(axis=1)

In [ ]:
true_max_L2 = data["n_items"].apply(lambda x: np.sqrt((x/3.0)*(x**2-1)))
data["max_L2_dist"] = data["max_L2_dist"] / true_max_L2
data["mean_L2_dist"] = data["mean_L2_dist"] / true_max_L2

In [ ]:
def read_P(string):
    return json.loads(string.replace("(", "[").replace(")", "]"))

def read_D(string):
    return np.array(json.loads(string))

def compute_from_D(string, func=np.sum):
    return func(read_D(string))

In [ ]:
# TODO (jwaschur): create train/test split for meaningful R_squared

def run_linear_model(X, y, verbose=False):
    if verbose:
        print("##############################")
        print("Linear model:")
        print("Predicting {} from {}".format(y.name, list(X.columns)))
    
    lm = LinearRegression()
    model = lm.fit(X,y)
    
    predictions = lm.predict(X)
    residuals = y - predictions
    R_squared = lm.score(X,y)
    coefficients = lm.coef_
    intercept = lm.intercept_
    
    # Get p-values for each of the parameters
    # Code found at https://stackoverflow.com/questions/27928275/find-p-value-significance-in-scikit-learn-linearregression
    params = np.append(intercept,coefficients)
    newX = pd.DataFrame({"Constant":np.ones(len(X))}).join(pd.DataFrame(X.reset_index(drop=True)))
    MSE = (sum((y-predictions)**2))/(len(newX)-len(newX.columns))
    var_b = MSE*(np.linalg.inv(np.dot(newX.T,newX)).diagonal())
    sd_b = np.sqrt(var_b)
    ts_b = params/ sd_b
    p_values =[2*(1-stats.t.cdf(np.abs(i),(len(newX)-len(newX.columns)-1))) for i in ts_b]
    
    # Plot the model with predictions if only one predictor
    if len(X.columns) == 1:
        col = X.columns[0]
        plt.scatter(X[col], y, alpha=0.1, color="b")
        plt.plot(X[col], predictions, color="r", label="y = {:.3E}*x + {:.3E}".format(coefficients[0], intercept))
        plt.xlabel(col)
        plt.ylabel(y.name)
        plt.legend()
        plt.title("Linear Model (R^2 = {:.3f}, p = {:.2E})".format(R_squared, p_values[1]))
        plt.show()
        plt.clf()
    else:
        print("\nR^2 = {:.3f}\n".format(R_squared))
        print("{:15}  {:8} (p-value)".format("Predictor", "Coeff"))
        print("---------------------------------------")
        print("{:15}: {:8.4f} (p = {:.2E})".format("Intercept", intercept, p_values[0]))
        for idx, col in enumerate(X.columns):
            print("{:15}: {:8.4f} (p = {:.2E})".format(col, coefficients[idx], p_values[idx+1]))
    
    if verbose:
        # Plot the histogram of residuals
        plt.hist(residuals, bins=30, density=True)
        plt.xlabel("Residual")
        plt.ylabel("Density")
        plt.title("Residual Density Histogram")
        plt.show()

        print("##############################\n")

In [ ]:
rankability_measures = ["kendall_w", "p_lowerbound", "max_L2_dist", "mean_L2_dist", "min_tau", "mean_tau", "k", "degree_of_linearity"]

for measure in rankability_measures:
    X = data[[measure]]
    y = data["overall_mean_sensitivity"]
    run_linear_model(X, y, verbose=True)

run_linear_model(data[rankability_measures],
                 data["overall_mean_sensitivity"],
                 verbose=True)

In [ ]:
for sensitivity in sensitivities:
    X = data[["k"]]
    y = data[sensitivity]
    run_linear_model(X, y, verbose=True)

run_linear_model(data[["k"]],
                 data["overall_mean_sensitivity"],
                 verbose=True)

In [ ]:
for measure in rankability_measures:
    if measure == "k":
        continue
    run_linear_model(data[[measure]], data["k"], verbose=True)

In [ ]:
data_temp_mask = (data["k"] > 55) & (data["k"] < 68)
data_temp = data.loc[data_temp_mask]
run_linear_model(data_temp[["k"]],
                 data_temp["overall_mean_sensitivity"],
                 verbose=True)

In [ ]:
def read_P(string):
    return json.loads(string.replace("(", "[").replace(")", "]"))

predicted_ps = []
for P_str in data["P_repeats"]:
    P_repeats = read_P(P_str)
    P_repeats = [str(r) for r in P_repeats]
    repeats = pd.Series(P_repeats).value_counts().value_counts()
    f1 = repeats.loc[1] if 1 in repeats.index else 0
    f2 = repeats.loc[2] if 2 in repeats.index else 0.5
    predicted_ps.append(repeats.sum() + f1**2/(2*f2))
predicted_ps = pd.Series(predicted_ps, name="predicted_p")
print(predicted_ps - data["p_lowerbound"])
run_linear_model(pd.DataFrame(predicted_ps), data["overall_mean_sensitivity"], verbose=True)